# Part V: Security and Authentication

## Chapter 12: OAuth2 and JWT

Building on the security fundamentals from Chapter 11, we now implement modern, stateless authentication using OAuth2 and JSON Web Tokens (JWT). This combination is the industry standard for API authentication, enabling scalable, secure access control without server-side session storage. FastAPI provides first-class support for OAuth2 through integrated utilities that automatically generate the interactive authorization UI in your documentation.

---

### 12.1 OAuth2 Password Flow: Understanding the Standard

The OAuth2 Password Flow (also called Resource Owner Password Credentials Grant) is designed for first-party applications where the client is trusted with user credentials. While OAuth2 Authorization Code flow is preferred for third-party apps, the Password flow remains common for mobile apps and single-page applications (SPAs) consuming their own API.

#### OAuth2 Password Flow Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│              OAuth2 Password Flow Authentication                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌─────────┐          ┌─────────┐          ┌─────────┐          │
│  │  Client │          │   API   │          │  Auth   │          │
│  │   (SPA) │          │ Server  │          │ Server  │          │
│  └────┬────┘          └────┬────┘          └────┬────┘          │
│       │                    │                    │                │
│       │  1. POST /token    │                    │                │
│       │  username=alice    │                    │                │
│       │  password=secret   │                    │                │
│       │───────────────────▶│                    │                │
│       │                    │  2. Validate     │                │
│       │                    │     credentials  │                │
│       │                    │───────────────────▶│                │
│       │                    │                    │                │
│       │                    │  3. User valid     │                │
│       │                    │◀───────────────────│                │
│       │                    │                    │                │
│       │  4. Return tokens  │                    │                │
│       │  {                 │                    │                │
│       │    access_token,   │                    │                │
│       │    refresh_token,  │                    │                │
│       │    token_type:     │                    │                │
│       │    "bearer"       │                    │                │
│       │  }                 │                    │                │
│       │◀───────────────────│                    │                │
│       │                    │                    │                │
│       │  5. GET /protected │                    │                │
│       │  Authorization:    │                    │                │
│       │  Bearer <token>   │                    │                │
│       │───────────────────▶│                    │                │
│       │                    │  6. Validate JWT   │                │
│       │                    │     signature      │                │
│       │                    │     & expiration   │                │
│       │                    │                    │                │
│       │  7. Return data    │                    │                │
│       │◀───────────────────│                    │                │
│       │                    │                    │                │
└───────┼────────────────────┼────────────────────┼────────────────┘
        │                    │                    │
Note:   │                    │                    │
- Tokens are JWTs (signed, not encrypted)          │
- Access tokens short-lived (15-30 min)            │
- Refresh tokens long-lived (7-30 days)             │
- All communication over HTTPS only                │
```

#### Implementing OAuth2 Password Flow

```python
# oauth2_implementation.py
from fastapi import FastAPI, Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer, OAuth2PasswordRequestForm
from pydantic import BaseModel
from datetime import datetime, timedelta
from typing import Optional
from jose import JWTError, jwt  # python-jose library
from passlib.context import CryptContext

# Configuration
SECRET_KEY = "your-secret-key-here-change-in-production"  # Use openssl rand -hex 32
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
REFRESH_TOKEN_EXPIRE_DAYS = 7

app = FastAPI(title="OAuth2 JWT Authentication")

# Password hashing (from Chapter 11)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# OAuth2 scheme - tells FastAPI where token URL is
# This creates the "Authorize" button in /docs
oauth2_scheme = OAuth2PasswordBearer(
    tokenUrl="token",
    scheme_name="JWT",
    description="OAuth2 Password Flow with JWT tokens",
)

# Data models
class Token(BaseModel):
    """Token response model."""
    access_token: str
    refresh_token: str
    token_type: str = "bearer"
    expires_in: int  # seconds

class TokenData(BaseModel):
    """Data stored inside JWT payload."""
    username: Optional[str] = None
    scopes: list[str] = []

class User(BaseModel):
    """User model (safe - no password)."""
    username: str
    email: Optional[str] = None
    full_name: Optional[str] = None
    disabled: bool = False
    scopes: list[str] = []

class UserInDB(User):
    """User with hashed password (internal use)."""
    hashed_password: str


# Mock database
fake_users_db = {
    "alice": {
        "username": "alice",
        "email": "alice@example.com",
        "full_name": "Alice Wonderland",
        "hashed_password": pwd_context.hash("secret123"),
        "disabled": False,
        "scopes": ["users:read", "items:read", "items:write"],
    },
    "bob": {
        "username": "bob",
        "email": "bob@example.com",
        "full_name": "Bob Builder",
        "hashed_password": pwd_context.hash("builder456"),
        "disabled": False,
        "scopes": ["users:read"],
    },
    "admin": {
        "username": "admin",
        "email": "admin@example.com",
        "full_name": "Admin User",
        "hashed_password": pwd_context.hash("adminsecret"),
        "disabled": False,
        "scopes": ["admin", "users:read", "users:write", "items:read", "items:write"],
    }
}


# Utility functions
def verify_password(plain_password: str, hashed_password: str) -> bool:
    """Verify password against hash."""
    return pwd_context.verify(plain_password, hashed_password)


def get_user(db: dict, username: str) -> Optional[UserInDB]:
    """Retrieve user from database."""
    if username in db:
        user_dict = db[username]
        return UserInDB(**user_dict)
    return None


def authenticate_user(db: dict, username: str, password: str) -> Optional[UserInDB]:
    """
    Authenticate user by username and password.
    
    Returns:
        UserInDB if credentials valid, None otherwise
    """
    user = get_user(db, username)
    if not user:
        return None
    if not verify_password(password, user.hashed_password):
        return None
    return user


def create_access_token(
    data: dict,
    expires_delta: Optional[timedelta] = None,
    token_type: str = "access"
) -> str:
    """
    Create JWT access token.
    
    Args:
        data: Payload data (usually contains "sub" for subject/username)
        expires_delta: Token lifetime. If None, uses default.
        token_type: "access" or "refresh"
    
    Returns:
        str: Encoded JWT token
    """
    to_encode = data.copy()
    
    # Set expiration
    if expires_delta:
        expire = datetime.utcnow() + expires_delta
    else:
        expire = datetime.utcnow() + timedelta(minutes=15)
    
    # Add claims
    to_encode.update({
        "exp": expire,  # Expiration time
        "iat": datetime.utcnow(),  # Issued at
        "type": token_type,  # Token type
    })
    
    # Encode with secret key
    encoded_jwt = jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)
    return encoded_jwt


# === AUTHENTICATION ENDPOINTS ===

@app.post("/token", response_model=Token)
async def login_for_access_token(
    form_data: OAuth2PasswordRequestForm = Depends()
):
    """
    OAuth2 token endpoint.
    
    Receives username/password from form data and returns JWT tokens.
    
    In Swagger UI (/docs):
    1. Click "Authorize"
    2. Enter username/password
    3. System automatically POSTs to this endpoint
    4. Token is stored and used for subsequent requests
    """
    # Authenticate user
    user = authenticate_user(fake_users_db, form_data.username, form_data.password)
    
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect username or password",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    if user.disabled:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="User account is disabled",
        )
    
    # Create access token with expiration
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    access_token = create_access_token(
        data={
            "sub": user.username,
            "scopes": form_data.scopes or user.scopes,  # Requested scopes or default
        },
        expires_delta=access_token_expires,
        token_type="access",
    )
    
    # Create refresh token (longer lived)
    refresh_token_expires = timedelta(days=REFRESH_TOKEN_EXPIRE_DAYS)
    refresh_token = create_access_token(
        data={"sub": user.username},
        expires_delta=refresh_token_expires,
        token_type="refresh",
    )
    
    return {
        "access_token": access_token,
        "refresh_token": refresh_token,
        "token_type": "bearer",
        "expires_in": ACCESS_TOKEN_EXPIRE_MINUTES * 60,
    }


@app.post("/token/refresh", response_model=Token)
async def refresh_access_token(refresh_token: str):
    """
    Refresh access token using refresh token.
    
    When access token expires, client sends refresh token to get new
    access token without re-entering credentials.
    """
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Invalid refresh token",
        headers={"WWW-Authenticate": "Bearer"},
    )
    
    try:
        # Decode refresh token
        payload = jwt.decode(refresh_token, SECRET_KEY, algorithms=[ALGORITHM])
        
        # Verify it's a refresh token
        if payload.get("type") != "refresh":
            raise credentials_exception
        
        username: str = payload.get("sub")
        if username is None:
            raise credentials_exception
        
    except JWTError:
        raise credentials_exception
    
    # Get user
    user = get_user(fake_users_db, username)
    if user is None or user.disabled:
        raise credentials_exception
    
    # Create new access token
    access_token_expires = timedelta(minutes=ACCESS_TOKEN_EXPIRE_MINUTES)
    new_access_token = create_access_token(
        data={"sub": user.username, "scopes": user.scopes},
        expires_delta=access_token_expires,
    )
    
    # Create new refresh token (token rotation for security)
    refresh_token_expires = timedelta(days=REFRESH_TOKEN_EXPIRE_DAYS)
    new_refresh_token = create_access_token(
        data={"sub": user.username},
        expires_delta=refresh_token_expires,
        token_type="refresh",
    )
    
    return {
        "access_token": new_access_token,
        "refresh_token": new_refresh_token,
        "token_type": "bearer",
        "expires_in": ACCESS_TOKEN_EXPIRE_MINUTES * 60,
    }
```

**Key Points about OAuth2 Implementation:**

1. **`OAuth2PasswordRequestForm`**: FastAPI provides this form class that automatically parses `username`, `password`, `scope`, `grant_type`, and `client_id` from form data.

2. **`OAuth2PasswordBearer`**: This class extracts the token from the `Authorization: Bearer <token>` header and validates its presence. It also generates the OpenAPI security scheme.

3. **Token URL**: The `tokenUrl="token"` parameter tells FastAPI where the token endpoint is located, enabling the interactive documentation to know where to POST credentials.

4. **Scope Parameter**: `OAuth2PasswordRequestForm` includes `scopes` which allows clients to request specific permissions during authentication.

---

### 12.2 JSON Web Tokens (JWT): Generating and Validating Tokens

JWTs are signed tokens containing claims (statements) about the user. They consist of three parts: Header (algorithm), Payload (data), and Signature (verification).

#### JWT Structure

```
┌─────────────────────────────────────────────────────────────────┐
│                     JSON Web Token Structure                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.                          │
│  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━                           │
│  Header (Base64URL encoded)                                      │
│  {                                                               │
│    "alg": "HS256",  ← Algorithm                                  │
│    "typ": "JWT"     ← Token type                                 │
│  }                                                               │
│                                                                  │
│  eyJzdWIiOiJhbGljZSIsImV4cCI6MTY4NDU2Nz...                      │
│  ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━                       │
│  Payload (Base64URL encoded)                                     │
│  {                                                               │
│    "sub": "alice",   ← Subject (username)                        │
│    "exp": 1684567890, ← Expiration (Unix timestamp)              │
│    "iat": 1684564290, ← Issued at                                │
│    "scopes": ["read", "write"], ← Custom claims                  │
│    "type": "access"  ← Token type                                │
│  }                                                               │
│                                                                  │
│  SflKxwRJSMeKKF2QT4fwpMe...                                      │
│  ━━━━━━━━━━━━━━━━━━━━━━━━━                                     │
│  Signature (HMACSHA256)                                          │
│  HMACSHA256(                                                     │
│    base64Url(header) + "." + base64Url(payload),                 │
│    secret_key                                                    │
│  )                                                               │
│                                                                  │
│  Important: JWTs are signed (integrity) but NOT encrypted.      │
│  Anyone can read the payload. Don't store secrets in JWTs.      │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

#### Complete JWT Validation Implementation

```python
# jwt_validation.py
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from jose import JWTError, jwt, ExpiredSignatureError
from pydantic import BaseModel
from datetime import datetime
from typing import Optional

# Configuration
SECRET_KEY = "09d25e094faa6ca2556c818166b7a9563b93f7099f6f0f4caa6cf63b88e8d3e7"
ALGORITHM = "HS256"

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

class TokenPayload(BaseModel):
    """Validated token payload."""
    sub: str  # Subject (username)
    exp: datetime  # Expiration
    iat: datetime  # Issued at
    scopes: list[str] = []
    type: str = "access"


async def decode_token(token: str) -> TokenPayload:
    """
    Decode and validate JWT token.
    
    Raises:
        HTTPException: 401 if token invalid, expired, or malformed
    
    Returns:
        TokenPayload: Decoded token data
    """
    try:
        # Decode token with verification
        payload = jwt.decode(
            token,
            SECRET_KEY,
            algorithms=[ALGORITHM],
            options={
                "verify_signature": True,
                "verify_exp": True,
                "verify_iat": True,
                "require": ["exp", "sub"],  # Required claims
            }
        )
        
        # Validate payload structure
        token_data = TokenPayload(**payload)
        
        # Additional validation: check token type
        if token_data.type != "access":
            raise HTTPException(
                status_code=status.HTTP_401_UNAUTHORIZED,
                detail="Invalid token type",
                headers={"WWW-Authenticate": "Bearer"},
            )
        
        return token_data
        
    except ExpiredSignatureError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Token has expired",
            headers={"WWW-Authenticate": "Bearer"},
        )
    except JWTError as e:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail=f"Could not validate credentials: {str(e)}",
            headers={"WWW-Authenticate": "Bearer"},
        )


# Dependency to get current user from token
async def get_current_user(token: str = Depends(oauth2_scheme)) -> dict:
    """
    Extract and validate user from JWT token.
    
    This dependency:
    1. Extracts token from Authorization header
    2. Validates signature and expiration
    3. Returns user data
    
    Usage:
        @app.get("/profile")
        async def profile(user: dict = Depends(get_current_user)):
            return user
    """
    token_data = await decode_token(token)
    
    # In real app: query database to get fresh user data
    # Don't trust all JWT data blindly
    user = get_user(fake_users_db, token_data.sub)
    
    if user is None:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="User not found",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    if user.disabled:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="User account is disabled",
        )
    
    return user


# Dependency with scopes checking
async def get_current_user_with_scopes(
    required_scopes: list[str],
    token: str = Depends(oauth2_scheme)
) -> dict:
    """
    Get current user and verify they have required scopes.
    
    Args:
        required_scopes: List of scopes required for this endpoint
    """
    token_data = await decode_token(token)
    
    # Check scopes
    token_scopes = set(token_data.scopes)
    required = set(required_scopes)
    
    if not required.issubset(token_scopes):
        missing = required - token_scopes
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail=f"Insufficient permissions. Missing scopes: {missing}",
        )
    
    user = get_user(fake_users_db, token_data.sub)
    if not user or user.disabled:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="User not found or disabled",
        )
    
    return user
```

#### Secure Token Storage (Cookies vs LocalStorage)

```python
# secure_token_storage.py
from fastapi import FastAPI, Response, Request, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from datetime import timedelta

app = FastAPI()

# Cookie-based token storage (more secure than localStorage)
@app.post("/token/cookie")
async def login_with_cookie(
    response: Response,
    form_data: OAuth2PasswordRequestForm = Depends()
):
    """
    Login that stores token in HTTP-only cookie.
    
    Benefits over localStorage:
    - HttpOnly: JavaScript cannot read token (XSS protection)
    - Secure: Only sent over HTTPS
    - SameSite: CSRF protection
    """
    user = authenticate_user(fake_users_db, form_data.username, form_data.password)
    if not user:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Incorrect credentials",
        )
    
    # Create token
    access_token_expires = timedelta(minutes=30)
    access_token = create_access_token(
        data={"sub": user.username},
        expires_delta=access_token_expires,
    )
    
    # Set cookie
    response.set_cookie(
        key="access_token",
        value=f"Bearer {access_token}",
        httponly=True,  # Cannot be accessed by JavaScript
        secure=True,    # Only sent over HTTPS
        samesite="lax", # CSRF protection
        max_age=1800,   # 30 minutes in seconds
        path="/",       # Available for all paths
    )
    
    return {"message": "Login successful"}


# Dependency to extract token from cookie
oauth2_scheme_optional = OAuth2PasswordBearer(tokenUrl="token", auto_error=False)

async def get_token_from_cookie_or_header(
    request: Request,
    token: str = Depends(oauth2_scheme_optional)
) -> str:
    """
    Extract token from header or cookie.
    
    Priority:
    1. Authorization header (Bearer token)
    2. access_token cookie
    """
    if token:
        return token
    
    # Check cookie
    cookie_token = request.cookies.get("access_token")
    if cookie_token:
        # Remove "Bearer " prefix if present
        if cookie_token.startswith("Bearer "):
            return cookie_token[7:]
        return cookie_token
    
    raise HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Not authenticated",
        headers={"WWW-Authenticate": "Bearer"},
    )


@app.get("/protected/cookie")
async def protected_with_cookie(
    token: str = Depends(get_token_from_cookie_or_header)
):
    """Endpoint that accepts token from cookie or header."""
    user = await get_current_user(token)
    return {"message": "Protected data", "user": user.username}


@app.post("/logout")
async def logout(response: Response):
    """
    Logout by clearing cookie.
    
    Note: For stateless JWT, server cannot invalidate tokens immediately.
    Token remains valid until expiration.
    Solutions:
    1. Short expiration times (15 min)
    2. Token blacklist (Redis) - check on each request
    3. Refresh token rotation
    """
    response.delete_cookie(
        key="access_token",
        httponly=True,
        secure=True,
        samesite="lax",
        path="/",
    )
    return {"message": "Logged out successfully"}
```

---

### 12.3 Dependency-Based Auth: Protecting Routes Using `Depends`

FastAPI's dependency injection system makes it easy to create reusable authentication dependencies that can be applied to any endpoint.

#### Creating Authentication Dependencies

```python
# auth_dependencies.py
from fastapi import Depends, HTTPException, status, Security
from fastapi.security import OAuth2PasswordBearer, SecurityScopes
from typing import Annotated  # Python 3.9+
import logging

logger = logging.getLogger(__name__)
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")

# Type alias for authenticated user
AuthenticatedUser = Annotated[dict, Depends(get_current_user)]


# Basic protection
@app.get("/users/me", response_model=User)
async def read_users_me(
    current_user: AuthenticatedUser
):
    """
    Get current user using dependency injection.
    
    FastAPI automatically:
    1. Extracts token from header
    2. Validates JWT
    3. Returns user dict
    4. Returns 401 if invalid
    """
    return current_user


# Optional authentication
async def get_optional_user(
    token: Annotated[str | None, Depends(oauth2_scheme)] = None
) -> dict | None:
    """
    Optional authentication - returns None if no token.
    
    Useful for endpoints that show different data based on auth status.
    """
    if not token:
        return None
    
    try:
        return await get_current_user(token)
    except HTTPException:
        return None


@app.get("/items/public")
async def read_items_public(
    current_user: Annotated[dict | None, Depends(get_optional_user)]
):
    """
    Public endpoint that shows more data if authenticated.
    """
    if current_user:
        return {
            "items": ["item1", "item2", "item3"],
            "user": current_user["username"],
            "personalized": True
        }
    return {
        "items": ["item1", "item2"],
        "user": None,
        "personalized": False
    }


# Multiple dependency layers
async def verify_token_active(
    token: Annotated[str, Depends(oauth2_scheme)]
) -> str:
    """
    Verify token is not blacklisted (check Redis/database).
    
    Additional layer beyond JWT validation.
    """
    # In production: check Redis blacklist
    # if await redis.exists(f"blacklist:{token}"):
    #     raise HTTPException(status_code=401, detail="Token revoked")
    
    # Check if user changed password after token issued
    # payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
    # user = get_user(fake_users_db, payload["sub"])
    # if payload["iat"] < user.password_changed_timestamp:
    #     raise HTTPException(status_code=401, detail="Password changed, please login again")
    
    return token


async def get_active_user(
    token: Annotated[str, Depends(verify_token_active)]
) -> dict:
    """Get user with additional verification."""
    return await get_current_user(token)


@app.get("/secure/data")
async def very_secure_data(
    user: Annotated[dict, Depends(get_active_user)]
):
    """Endpoint with additional token verification."""
    return {"secret": "data", "user": user["username"]}
```

---

### 12.4 Current User Pattern: Fetching the Authenticated User Globally

The "Current User" pattern involves creating a dependency that extracts the user from the token and makes it available to endpoints. This is the standard approach in FastAPI applications.

#### Complete Current User Implementation

```python
# current_user_pattern.py
from fastapi import Depends, HTTPException, status
from fastapi.security import OAuth2PasswordBearer
from sqlalchemy.ext.asyncio import AsyncSession
from sqlalchemy import select
from typing import Annotated

from app.database import get_db
from app.models import User as UserModel
from app.schemas import User as UserSchema

oauth2_scheme = OAuth2PasswordBearer(tokenUrl="token")


async def get_current_user(
    token: Annotated[str, Depends(oauth2_scheme)],
    db: Annotated[AsyncSession, Depends(get_db)]
) -> UserSchema:
    """
    Get current authenticated user from database.
    
    This dependency:
    1. Extracts JWT from Authorization header
    2. Validates token signature and expiration
    3. Queries database for fresh user data
    4. Returns full user model
    
    Usage:
        @app.get("/profile")
        async def profile(user: UserSchema = Depends(get_current_user)):
            return user
    """
    credentials_exception = HTTPException(
        status_code=status.HTTP_401_UNAUTHORIZED,
        detail="Could not validate credentials",
        headers={"WWW-Authenticate": "Bearer"},
    )
    
    try:
        # Decode JWT
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        user_id: str = payload.get("sub")
        
        if user_id is None:
            raise credentials_exception
            
    except JWTError:
        raise credentials_exception
    
    # Query database for fresh user data
    result = await db.execute(
        select(UserModel).where(UserModel.id == user_id)
    )
    user = result.scalar_one_or_none()
    
    if user is None:
        raise credentials_exception
    
    if not user.is_active:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Inactive user",
        )
    
    return UserSchema.model_validate(user)


async def get_current_active_user(
    current_user: Annotated[UserSchema, Depends(get_current_user)]
) -> UserSchema:
    """
    Dependency that ensures user is active (not disabled).
    
    Additional check beyond authentication.
    """
    if not current_user.is_active:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail="Inactive user",
        )
    return current_user


# Usage in routers
from fastapi import APIRouter

router = APIRouter()

@router.get("/me", response_model=UserSchema)
async def read_current_user(
    current_user: Annotated[UserSchema, Depends(get_current_active_user)]
):
    """
    Get current user profile.
    
    The current_user is the full database model with all fields.
    """
    return current_user


@router.put("/me", response_model=UserSchema)
async def update_current_user(
    user_update: UserUpdate,
    current_user: Annotated[UserSchema, Depends(get_current_active_user)],
    db: Annotated[AsyncSession, Depends(get_db)]
):
    """
    Update current user profile.
    
    Users can only update their own profile.
    """
    # Update logic here
    for field, value in user_update.model_dump(exclude_unset=True).items():
        setattr(current_user, field, value)
    
    await db.commit()
    await db.refresh(current_user)
    
    return current_user
```

---

### 12.5 Role-Based Access Control (RBAC): Scopes and Permissions

RBAC restricts system access based on user roles. FastAPI implements this through OAuth2 scopes, which represent permissions in the token.

#### Implementing RBAC with Scopes

```python
# rbac_implementation.py
from fastapi import Depends, HTTPException, status, Security
from fastapi.security import OAuth2PasswordBearer, SecurityScopes
from typing import Annotated

oauth2_scheme = OAuth2PasswordBearer(
    tokenUrl="token",
    # Define available scopes for documentation
    scopes={
        "users:read": "Read user information",
        "users:write": "Create or update users",
        "users:delete": "Delete users",
        "items:read": "Read items",
        "items:write": "Create or update items",
        "items:delete": "Delete items",
        "admin": "Full administrative access",
    },
)


class TokenData(BaseModel):
    username: str | None = None
    scopes: list[str] = []


async def get_current_user_with_scopes(
    security_scopes: SecurityScopes,
    token: Annotated[str, Depends(oauth2_scheme)]
) -> dict:
    """
    Authenticate user and verify required scopes.
    
    Args:
        security_scopes: Automatically injected scopes required by endpoint
        token: JWT token from header
    
    Raises:
        HTTPException: 401 if token invalid, 403 if insufficient scopes
    """
    # Authenticate token
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("sub")
        token_scopes = payload.get("scopes", [])
        
        if username is None:
            raise HTTPException(
                status_code=status.HTTP_401_UNAUTHORIZED,
                detail="Invalid authentication credentials",
                headers={"WWW-Authenticate": "Bearer"},
            )
            
        token_data = TokenData(username=username, scopes=token_scopes)
        
    except JWTError:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="Invalid authentication credentials",
            headers={"WWW-Authenticate": "Bearer"},
        )
    
    # Get user from database
    user = get_user(fake_users_db, token_data.username)
    if user is None:
        raise HTTPException(
            status_code=status.HTTP_401_UNAUTHORIZED,
            detail="User not found",
        )
    
    # Check scopes
    # security_scopes.scopes contains scopes required by endpoint decorator
    if security_scopes.scopes:
        # Check if user has admin scope (bypasses specific checks)
        if "admin" in token_data.scopes:
            return user
        
        # Check required scopes
        for scope in security_scopes.scopes:
            if scope not in token_data.scopes:
                raise HTTPException(
                    status_code=status.HTTP_403_FORBIDDEN,
                    detail=f"Not enough permissions. Required: {security_scopes.scope_str}",
                    headers={"WWW-Authenticate": f'Bearer scope="{security_scopes.scope_str}"'},
                )
    
    return user


# Helper dependency factories
def require_scopes(*scopes: str):
    """
    Factory to create scope requirements.
    
    Usage:
        @app.get("/admin", dependencies=[Depends(require_scopes("admin"))])
    """
    async def checker(
        user: Annotated[dict, Security(get_current_user_with_scopes, scopes=list(scopes))]
    ):
        return user
    return checker


# Protected endpoints with different permission levels

@app.get("/users/", dependencies=[Depends(require_scopes("users:read", "admin"))])
async def read_users():
    """
    List all users.
    
    Requires: users:read OR admin scope
    """
    return list(fake_users_db.values())


@app.post("/users/", dependencies=[Depends(require_scopes("users:write", "admin"))])
async def create_user(user: UserCreate):
    """
    Create new user.
    
    Requires: users:write OR admin scope
    """
    # Creation logic
    return {"message": "User created"}


@app.delete("/users/{username}", dependencies=[Depends(require_scopes("users:delete", "admin"))])
async def delete_user(username: str):
    """
    Delete user.
    
    Requires: users:delete OR admin scope
    """
    if username in fake_users_db:
        del fake_users_db[username]
    return {"message": "User deleted"}


@app.get("/items/")
async def read_items(
    user: Annotated[dict, Security(get_current_user_with_scopes, scopes=["items:read"])]
):
    """
    Read items with automatic scope checking.
    
    Security() injects required scopes into dependency.
    """
    return [{"item_id": "1", "name": "Item 1"}]


@app.post("/items/")
async def create_item(
    item: dict,
    user: Annotated[dict, Security(get_current_user_with_scopes, scopes=["items:write"])]
):
    """Create item."""
    return {"message": "Item created", "item": item}


# Admin only endpoint
@app.get("/admin/dashboard")
async def admin_dashboard(
    user: Annotated[dict, Security(get_current_user_with_scopes, scopes=["admin"])]
):
    """
    Admin dashboard - requires admin scope.
    
    Admin scope implies all other permissions.
    """
    return {
        "message": "Admin access granted",
        "user": user["username"],
        "stats": {
            "total_users": len(fake_users_db),
            "system_status": "healthy",
        },
    }


# Ownership-based permissions (user can only access own resources)
@app.get("/users/{user_id}/profile")
async def read_user_profile(
    user_id: str,
    current_user: Annotated[dict, Security(get_current_user_with_scopes, scopes=["users:read"])]
):
    """
    Read user profile with ownership check.
    
    Users can read their own profile, or any profile if admin.
    """
    # Check ownership or admin
    if current_user["username"] != user_id and "admin" not in current_user.get("scopes", []):
        raise HTTPException(
            status_code=status.HTTP_403_FORBIDDEN,
            detail="Not authorized to access this profile",
        )
    
    user = get_user(fake_users_db, user_id)
    if not user:
        raise HTTPException(status_code=404, detail="User not found")
    
    return user
```

#### Advanced RBAC with Hierarchical Roles

```python
# hierarchical_rbac.py
from enum import Enum
from typing import List

class Role(str, Enum):
    """Role hierarchy from least to most privileged."""
    USER = "user"
    MODERATOR = "moderator"
    ADMIN = "admin"
    SUPERUSER = "superuser"

# Permission mapping: role -> scopes
ROLE_SCOPES = {
    Role.USER: ["items:read", "users:read:own"],
    Role.MODERATOR: ["items:read", "items:write", "users:read", "users:write:own"],
    Role.ADMIN: ["items:read", "items:write", "items:delete", "users:read", "users:write", "users:delete"],
    Role.SUPERUSER: ["admin", "*"],  # Wildcard for all permissions
}

def get_scopes_for_role(role: Role) -> List[str]:
    """Get all scopes for a role including inherited ones."""
    scopes = set()
    
    # Accumulate scopes based on hierarchy
    if role == Role.USER:
        scopes.update(ROLE_SCOPES[Role.USER])
    elif role == Role.MODERATOR:
        scopes.update(ROLE_SCOPES[Role.USER])
        scopes.update(ROLE_SCOPES[Role.MODERATOR])
    elif role == Role.ADMIN:
        scopes.update(ROLE_SCOPES[Role.USER])
        scopes.update(ROLE_SCOPES[Role.MODERATOR])
        scopes.update(ROLE_SCOPES[Role.ADMIN])
    elif role == Role.SUPERUSER:
        scopes.add("*")  # All permissions
    
    return list(scopes)


def has_permission(user_scopes: List[str], required_scope: str) -> bool:
    """Check if user has permission (supports wildcards)."""
    if "*" in user_scopes or "admin" in user_scopes:
        return True
    
    if required_scope in user_scopes:
        return True
    
    # Check wildcard patterns (e.g., "users:*" matches "users:read")
    scope_parts = required_scope.split(":")
    for i in range(len(scope_parts), 0, -1):
        wildcard = ":".join(scope_parts[:i]) + ":*"
        if wildcard in user_scopes:
            return True
    
    return False


# Usage in token creation
def create_token_with_role(username: str, role: Role) -> str:
    """Create token including role-based scopes."""
    scopes = get_scopes_for_role(role)
    return create_access_token(
        data={"sub": username, "scopes": scopes, "role": role.value}
    )
```

---

### Summary

In this chapter, you implemented modern token-based authentication:

1. **OAuth2 Password Flow**: Using `OAuth2PasswordBearer` and `OAuth2PasswordRequestForm` to handle username/password exchanges for access tokens, with automatic interactive documentation support.

2. **JWT Implementation**: Creating and validating signed tokens with `python-jose`, handling expiration, token refresh mechanisms, and secure cookie storage.

3. **Dependency-Based Auth**: Creating reusable `Depends()` patterns that extract and validate tokens automatically, enabling clean endpoint definitions with `current_user: User = Depends(get_current_user)`.

4. **Current User Pattern**: Fetching authenticated user data from the database on each request while keeping tokens stateless, ensuring fresh user data and permission checks.

5. **RBAC with Scopes**: Implementing fine-grained permissions using OAuth2 scopes, checking permissions with `Security()` dependencies, and supporting hierarchical roles.

**Security Best Practices Implemented:**
- Short-lived access tokens (15-30 minutes) with refresh tokens
- HTTPS-only cookie storage with HttpOnly and SameSite attributes
- Scope-based authorization preventing privilege escalation
- Token validation including signature, expiration, and type checking
- Separation of access tokens (short) and refresh tokens (long)

---

### What's Next?

**Chapter 13: SQL Databases with SQLAlchemy** will cover:
- **Async SQLAlchemy**: Setting up async engines and sessions for non-blocking database operations
- **Dependency Injection for DB Sessions**: Managing database connections per request with proper lifecycle handling
- **CRUD Operations**: Creating, reading, updating, and deleting data with async SQLAlchemy
- **Relationships**: Handling foreign keys and relationships in API responses while avoiding N+1 queries
- **Database Integration**: Combining authentication from this chapter with database-backed user storage

This next chapter bridges the gap between in-memory authentication examples and production database integration, enabling you to build scalable, data-driven APIs.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='11. security_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../6. database_integration/13. sql_databases_with_sqlalchemy.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
